In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
# Import models
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from explainerdashboard import RegressionExplainer, ExplainerDashboard
import xarray as xr

data_dir = "data/all_files_zipped/" # Define data path




Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [3]:
from zipfile import ZipFile
import os

mypath = 'data\\rainfall'
if not os.path.isdir(mypath):
   os.makedirs(mypath)


def get_all_file_paths(directory):
  
    # initializing empty file paths list
    file_paths = []
  
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    # returning all file paths
    return file_paths    

paths = get_all_file_paths('data\\all_files_zipped')
paths

for file_name in paths:
    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        # printing all the contents of the zip file
        zip.printdir()

        # extracting all the files
        print('Extracting all the files now...')
        zip.extractall(path=mypath)
        print('Done!')


# Open the CHIRPS NetCDF file






File Name                                             Modified             Size
2000.nc                                        2023-09-25 14:50:44     70325541
Extracting all the files now...
Done!
File Name                                             Modified             Size
2001.nc                                        2023-09-25 14:50:44     70133723
Extracting all the files now...
Done!
File Name                                             Modified             Size
2002.nc                                        2023-09-25 14:50:44     70133905
Extracting all the files now...
Done!
File Name                                             Modified             Size
2003.nc                                        2023-09-25 14:50:44     70134087
Extracting all the files now...
Done!
File Name                                             Modified             Size
2004.nc                                        2023-09-25 14:50:44     70326269
Extracting all the files now...
Done!
File Name 

In [4]:
# Code to set the data to a correct dataframe with columns!!
import numpy as np
import xarray as xr

# DataFrame has lon, lat and time. The rainfall(pr) value has for every combination of lon, lat and time, pr has a value.
# There are 320 values for lat, for every lon value.
ds = xr.open_dataset('data\\rainfall\\2000.nc')
ds['pr']

val = np.arange(35136000).reshape(366,320,300)
time = np.array(ds['time'])
lat = np.array(ds['lat'])
lon = np.array(ds['lon'])

da = xr.DataArray(
    val,
    coords={'time': time,
        'lat': lat,
        'lon': lon},
    dims=["time","lat","lon"]
)

DataFrame = da.to_dataframe(name='value').reset_index()
pr_data = np.array(ds['pr'])
reshaped_data = np.reshape(pr_data, 35136000)
reshaped_data

DataFrame['value'] = reshaped_data.tolist()
DataFrame




,time,lat,lon,value
0,2000-01-01 12:00:00,-39.875,-19.875,NaN
1,2000-01-01 12:00:00,-39.875,-19.625,NaN
2,2000-01-01 12:00:00,-39.875,-19.375,NaN
3,2000-01-01 12:00:00,-39.875,-19.125,NaN
4,2000-01-01 12:00:00,-39.875,-18.875,NaN
...,...,...,...,...
35135995,2000-12-31 12:00:00,39.875,53.875,2.0
35135996,2000-12-31 12:00:00,39.875,54.125,1.0
35135997,2000-12-31 12:00:00,39.875,54.375,1.0
35135998,2000-12-31 12:00:00,39.875,54.625,2.0


In [ ]:
#cluster data on the countie points 


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from explainerdashboard import RegressionExplainer, ExplainerDashboard

DataFrame.dropna(subset=['value'], inplace=True)
DataFrame
X = DataFrame[['lon', 'lat', 'value']] 
y = DataFrame[['value']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train.values.ravel())
explainer = RegressionExplainer(model, X_test, y_test)
#Start the Dashboard
db = ExplainerDashboard(explainer,title="Diabetes Prediction",whatif=False)
db.run()

(14442945, 3) (14442945, 1) (3610737, 3) (3610737, 1)
Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...


MemoryError: Unable to allocate 23.5 GiB for an array with shape (3610737, 4, 218) and data type float64